# Livrable final du projet - Projet Green Graph
*Equipe CesiCDP - Chef de projet : Leila | Opérateurs : Tom, Edwin*</br>
## Sommaire :
    1. Introduction
    2. Définition du problème 
        2.1. Description du problème de tournée de livraison
        2.2. Contraintes
        2.3. Objectifs d’optimisation
    3. Modélisation formelle
        3.1. Hypothèses générales
        3.2. Représentation formelle du réseau
        3.3. Présentation des contraintes supplémentaires retenues
        3.4. Représentation formelle des contraintes choisies
    4. Analyse de la complexité
        4.1. Classification du problème
        4.2. Impact des contraintes supplémentaires sur la complexité
    5. Plan de Travail et Organisation du Projet
        5.1. Étapes prévues 
        5.2. Répartition des tâches dans l’équipe
        5.3. Outils utilisés 
    6. Conclusion
    7. Annexes
        7.1. Glossaire des termes techniques
        7.2. Références bibliographiques